In [ ]:
%config IPCompleter.greedy=True
import numpy as np
import torch 
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

In [ ]:
# Load predefined alexnet model
model = models.alexnet(num_classes = 10)
model.cuda()

# Create image transformation
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(244),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [ ]:
imagenet_data_train = datasets.CIFAR10('./data', download = True, train = True, transform = transform_train)
imagenet_data_test = datasets.CIFAR10('./data', download = True, train = False, transform = transform_test)

In [ ]:
# Create Training Hyperparameters
NUM_EPOCHS = 50
BATCH_SIZE = 128
NUM_CLASSES = 10
LEARNING_RATE = 0.01
MOMENTUM = 0.9
TRAINING_RECORD_INTERVAL = 25

train_loader = torch.utils.data.DataLoader(imagenet_data_train, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=0)

test_loader = torch.utils.data.DataLoader(imagenet_data_test, batch_size=BATCH_SIZE,
                                          shuffle=False, num_workers=0)

criterion = torch.nn.CrossEntropyLoss().cuda()
optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=0.0005)

training_accuracy = []
testing_accuracy = []

cudnn.benchmark = True

for epoch_idx in range(NUM_EPOCHS):
    
    for idx, (inputs, labels) in enumerate(train_loader):
        
        inputs, labels = inputs.cuda(), labels.cuda()
        optimizer.zero_grad()

        outputs = model(inputs)
        
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        loss = criterion(outputs, labels)
        
        
        if idx % TRAINING_RECORD_INTERVAL == 0:
          training_accuracy.append(100 * correct / BATCH_SIZE)

          print('Epoch: %d\tIterval: %d\tAccuracy : %d%%' % (
              epoch_idx,
              idx,
              100 * correct / BATCH_SIZE))
        
        loss.backward()

        optimizer.step()

    testing_correct = 0
    for idx, (inputs, labels) in enumerate(test_loader):

        inputs, labels = inputs.cuda(), labels.cuda()
        outputs = model(inputs)

        _, predicted = torch.max(outputs.data, 1)
        testing_correct += (predicted == labels).sum().item()
        
    print('Epoch: %d\tAccuracy: %d %%' % (epoch_idx, 100 * testing_correct / len(imagenet_data_test)))
    testing_accuracy.append(100 * testing_correct / len(imagenet_data_test))
        
np.save('/content/drive/My Drive/Colab Notebooks/Results/Control/training_accuracy.npy', training_accuracy)
np.save('/content/drive/My Drive/Colab Notebooks/Results/Control/testing_accuracy.npy', testing_accuracy)
torch.save(model.state_dict(), '/content/drive/My Drive/Colab Notebooks/Results/Control/model_control.pt')